## Notes on Minnow

This notebook collects some notes on how the pulsar timing code [minnow](https://github.com/meyers-academic/minnow/tree/main) works, based on helpful input and guidance from Pat Meyers. Some of the components of `minnow` will be essential if we are to run the Kalman state-space algorithms on real PTA data with `Argus`

For all that follows we will assume that there is just a single pulsar, $N_{\rm psr} =1$. The extension to general $N_{\rm psr}$ should be straightforward. 

For the purposes of running state-space algorithms, we need to define the following:

* $\boldsymbol{X}$ : the state vector, dimension $n_X$
* $\boldsymbol{Y}$ : the observation vector, dimension $n_Y$
* $\boldsymbol{F}$ : the state transition matrix, dimension $n_X$ $\times$ $n_X$
* $\boldsymbol{H}$ : the measurement matrix, dimension $n_Y$ $\times$ $n_X$
* $\boldsymbol{Q}$ : the process noise covariance matrix, dimension $n_X$ $\times$ $n_X$
* $\boldsymbol{R}$ : the measurement noise covariance matrix, dimension $n_Y$ $\times$ $n_Y$

### 1. Observations

The 'raw' pulsar data comes in the form of a `.toa` file and a `.par` file.

The `.toa` gives you the pulse times of arrival, the `.par` file gives the best guess of some of the pulsar parameters such as its spin frequency, position on the sky, etc. Note that these parameters are very well known a-priori. 

The `.toa` and the `.par` get passed through a timing software like `TEMPO` or `PINT` to produce **timing residuals**

**Question 1.1: I am not eactly clear on the how different observation frequency bands come into play here, but I don't think it is a problem for us. One can imagine getting multiple `.toa` files are different observation frequencies, and producing different timing residuals**.

For now I am just going to use the heuristic that there is a single data file that holds TOAs and timing residuals at different frequencies. This seems to be how the `.feather` data files are structured in `minnow`, see e.g. code block below 


In [9]:
import pyarrow.feather as feather
df = feather.read_feather('../data/data_from_minnow/v1p1_de440_pint_bipm2019-B1855+09.feather')
df


,toas,stoas,toaerrs,residuals,freqs,backend_flags,Mmat_0,Mmat_1,Mmat_2,Mmat_3,...,flags_wt,flags_flux,flags_fluxe,flags_proc,flags_pta,flags_ver,flags_to,flags_pout_gibbs,flags_clkcorr,flags_simul
0,4.610194e+09,4.610194e+09,1.212000e-06,-1.570598e-06,1386.037543,L-wide_ASP,0.005362,1.318638e+06,-1.621386e+14,-1.0,...,2.6903,3.25644,0.021,15y,NANOGrav,2021.08.25-9d8d617,-0.839e-6,0.012491908440860854,2.630754008821355e-05,
1,4.610194e+09,4.610194e+09,1.017600e-05,-1.715160e-06,1410.038193,L-wide_ASP,0.005362,1.318638e+06,-1.621386e+14,-1.0,...,1.086,1.02708,0.053,15y,NANOGrav,2021.08.25-9d8d617,-0.839e-6,0.012905991732230643,2.6307540088161908e-05,
2,4.610194e+09,4.610194e+09,1.705000e-06,-8.360033e-07,1422.038518,L-wide_ASP,0.005362,1.318638e+06,-1.621386e+14,-1.0,...,2.5761,2.44634,0.023,15y,NANOGrav,2021.08.25-9d8d617,-0.839e-6,0.012457192575780427,2.630754008813714e-05,
3,4.610194e+09,4.610194e+09,9.520000e-07,-5.140002e-07,1414.038302,L-wide_ASP,0.005362,1.318638e+06,-1.621386e+14,-1.0,...,2.8314,4.10634,0.022,15y,NANOGrav,2021.08.25-9d8d617,-0.839e-6,0.01234654947538391,2.630754008815364e-05,
4,4.610194e+09,4.610194e+09,1.968000e-06,-2.606156e-08,1402.037977,L-wide_ASP,0.005362,1.318638e+06,-1.621386e+14,-1.0,...,2.6525,2.10044,0.022,15y,NANOGrav,2021.08.25-9d8d617,-0.839e-6,0.01288848991589095,2.6307540088178913e-05,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7753,5.102021e+09,5.102021e+09,2.059000e-06,8.940844e-07,1722.979881,L-wide_PUPPI,0.005362,-1.318589e+06,-1.621266e+14,-1.0,...,15.068,0.673438,0.0079,15y,NANOGrav,2021.08.25-9d8d617,,0.01296979848578532,2.776931610305224e-05,
7754,5.102021e+09,5.102021e+09,6.560000e-07,1.781750e-06,1686.731252,L-wide_PUPPI,0.005362,-1.318589e+06,-1.621266e+14,-1.0,...,11.499,3.57581,0.013,15y,NANOGrav,2021.08.25-9d8d617,,0.012244980649401821,2.776931610308031e-05,
7755,5.102021e+09,5.102021e+09,1.982000e-06,2.220510e-06,1674.821045,L-wide_PUPPI,0.005362,-1.318589e+06,-1.621266e+14,-1.0,...,10.409,1.12112,0.013,15y,NANOGrav,2021.08.25-9d8d617,,0.012503069874448748,2.7769316103089936e-05,
7756,5.102021e+09,5.102021e+09,6.450000e-07,1.934482e-06,1749.682344,L-wide_PUPPI,0.005362,-1.318589e+06,-1.621266e+14,-1.0,...,8.843,4.59824,0.017,15y,NANOGrav,2021.08.25-9d8d617,,0.012712839123538945,2.7769316103032728e-05,


The observation $\boldsymbol{Y}$ vector is a vector of timing residuals $\delta t$ where the dimension $n_Y$ is set by the number of observation frequency bands, e.g. 

$$\boldsymbol{Y} = \left [\delta t(\nu_1),\delta t(\nu_2),\delta t(\nu_3),\dots \right ]$$

Note that at a given timestep there may be missing data at a particular frequency band, but the Kalman filter can handle this no problem. Similarly, different pulsars may be observed at different times, but this can also be handled by the algorithm straightforwardly. 

`minnow` pre-processes the `.feather` files, adding some zero-padding, https://github.com/meyers-academic/minnow/blob/main/src/minnow/pulsar.py

### 2. Hidden states and transition matrix

In `minnow`, the hidden state variables are 

$$\boldsymbol{X} = \left[\delta \phi, \delta f, \delta \dot{f} \right] $$

i.e. the **deviations from the spin-down parameters in the timing model fit**.

These variables evolve as,

$$ \frac{d \, \delta \phi}{dt} = \delta f$$

$$ \frac{d \, \delta f}{dt} = \delta \dot{f}$$

$$ \frac{d \, \delta \dot{f}}{dt} = 0 $$


or, taking $\frac{d\boldsymbol{X}}{dt} = \boldsymbol{A} \boldsymbol{X}$, then 

$$\boldsymbol{A} = \begin{pmatrix}0 & 1 & 0 \\\ 0 & 0 &1 \\\ 0 & 0 & 0\end{pmatrix} $$

We can compute the transition matrix $\boldsymbol{F}$ by discretising in the usual way, i.e. $\boldsymbol{F} = \exp \left( \boldsymbol{A} \Delta t \right)$, where $\Delta t$ is the time difference between two consecutive TOAs, 

$$\boldsymbol{F} = \begin{pmatrix}1 & \Delta t & \Delta t^2/2  \\\ 0 & 1 &\Delta t \\\ 0 & 0 & 1\end{pmatrix} $$

This matrix is equivalent to that defined in https://github.com/meyers-academic/minnow/blob/main/src/minnow/signals.py#L87 

### 3. Measurement matrix 

The measurement matrix $\boldsymbol{H}$ relates the states and the observations linearly, $\boldsymbol{Y} = \boldsymbol{H} \boldsymbol{X}$ (+ noise).

The residuals $\delta t$ are related to the phase residuals in the state as $\delta t = \delta \phi / f_0$ where $f_0$ is the pulsar rotation frequency.

**Question 3.1: I guess we can just read $f_0$ directly from the fit?**.

For the vaccum case where we ignore dispersion (more of that later) and the observation vector collapses to $\boldsymbol{Y} = \left [\delta t(\nu)\right ]$, then $\boldsymbol{H}$ is trivially 


$$\boldsymbol{H} = \left[ f_0^{-1} \right] $$

### 4. More complicated models 

The above is a good starting place, but what if we also want to include additional components of the timing model, or dispersion measure corrections? 




#### 4.1 Dispersion measure

The influence of the ISM on the radio propagation induces an additional delay in $\Delta t_{\rm DM}$ in the TOAs. The delay for an observation at frequency $\nu_i$ is

$$\Delta t_{\rm DM} = k \, \text{DM} \,  \nu_i^2 $$

see e.g. Equation 1 of https://www.aanda.org/articles/aa/full_html/2020/12/aa39517-20/aa39517-20.html


**Question 4.1. Are dispersion corrections not ALREADY included as part of the PINT/TEMPO timing model? Is this better though of as corrections to the DM?**

We augment the state space to include the dispersion measure

$$\boldsymbol{X} = \left[\delta \phi, \delta f, \delta \dot{f}, \text{DM} \right] $$

where the DM is constant in time such that 

$$\boldsymbol{A} = \begin{pmatrix}0 & 1 & 0 &0 \\\ 0 & 0 &1&0 \\\ 0 & 0 & 0&0 \\\ 0 & 0 & 0 & 0\end{pmatrix} $$

and

$$\boldsymbol{F} = \begin{pmatrix}1 & \Delta t & \Delta t^2/2 &0  \\\ 0 & 1 &\Delta t &0 \\\ 0 & 0 & 1&0  \\\ 0 & 0 & 0&1 \end{pmatrix} $$

This matrix is equivalent to that defined in https://github.com/meyers-academic/minnow/blob/main/src/minnow/signals.py#L7 


The $\boldsymbol{H}$ matrix can be updated correspondingly. We now use the full observation vector $$\boldsymbol{Y} = \left [\delta t(\nu_1),\delta t(\nu_2),\delta t(\nu_3),\dots \right ]$$ and $\boldsymbol{H}$ becomes


$$\boldsymbol{H} = \begin{pmatrix} f_0^{-1}  & 0 & 0 &k\nu_1 ^{-2} \\\ f_0^{-1}  & 0 & 0 &k\nu_2 ^{-2} \\\ f_0^{-1}  & 0 & 0 &k\nu_3 ^{-2}  \\\ \vdots \end{pmatrix}$$

This is equivalent to https://github.com/meyers-academic/minnow/blob/main/src/minnow/signals.py#L53 (ignoring the `Mmats` corrections which come in the next section). 

Writing this out explicitly, we are claiming that the timing residual at a particular observation frequency is

$$ \delta t \left( \nu_i \right) = \frac{\delta \phi}{f_0} + k \, \text{DM} \, \nu_i^2$$



#### 4.2 Full timing model 

The full timing model corrections are included as part of the "design matrix" $\boldsymbol{M}$ (see e.g. Section 7 of https://arxiv.org/abs/2105.13270).

The timing model fit returns a bunch of derivatives of the TOAs with respect to each timing ephemeris parameter. In the `.feather` data files there are a bunch of columns called `MMat_*` which hold the partial derivatives of the TOAs with respect to each timing-ephemeris parameter.

To include this, we proceed analogously to the dispersion measure case, namely augment the state-space and update the measurement equation. Specifically,

$$\boldsymbol{X} = \left[\delta \phi, \delta f, \delta \dot{f}, \text{DM},\text{M1},\text{M2},\dots, \right] $$

$$\boldsymbol{F} = \begin{pmatrix} \boldsymbol{F}_{\phi} & 0  \\\ 0 & \boldsymbol{I} \end{pmatrix} $$


where $\boldsymbol{F}_{\phi}$ is a block matrix, dimension 3x3, for $\delta \phi, \delta f, \delta \dot{f}$, as defined in Section 2, and $\boldsymbol{I}$ is an identity matrix, with dimension set by the DM plus the number of $M$ terms. This matrix is equivalent to that defined in https://github.com/meyers-academic/minnow/blob/main/src/minnow/signals.py#L7 


The $\boldsymbol{H}$ matrix is then 


$$\boldsymbol{H} = \begin{pmatrix} f_0^{-1}  & 0 & 0 &k\nu_1 ^{-2} & 1, & 1, \dots \\\ f_0^{-1}  & 0 & 0 &k\nu_2 ^{-2} & 1, & 1, \dots\\\ f_0^{-1}  & 0 & 0 &k\nu_3 ^{-2} & 1 & 1, \dots \\\ \vdots \end{pmatrix}$$

**Note** this is a little different to https://github.com/meyers-academic/minnow/blob/main/src/minnow/signals.py#L53, where the $M_i$ parameters are included in the measurement matrix. I think this makes sense if instead the state is $\boldsymbol{X} = \left[\delta \phi, \delta f, \delta \dot{f}, \text{DM},1,1,\dots, \right]$. Since the DM and $M_i$ parameters are constant, I don't think it matters too much whether they "live" in $\boldsymbol{X}$ or in $\boldsymbol{H}$. 

Writing this out explicitly, we are claiming that the timing residual at a particular observation frequency is

$$ \delta t \left( \nu_i \right) = \frac{\delta \phi}{f_0} + k \, \text{DM} \, \nu_i^2 + M_1 + M_2 + \dots $$








### 5. Additional comments and questions

1. I don't think the `.feather` data files, specifically the `residuals` column, should be thought of as the observations $\boldsymbol{Y}$. In `minnow` there is a [pre-processing step](https://github.com/meyers-academic/minnow/blob/main/src/minnow/pulsar.py) which uses `discovery` to load the pulsar object. I do not have access to `discovery` so it is a little hard to play with this and understand better what is going on, but [in the unit tests](https://github.com/meyers-academic/minnow/blob/main/tests/test_pulsar.py) the number of residuals is suggested to be equal to 360, which is $\ll$ number of rows in the `.feather` data file.

2. Is the description of the full timing model corrections $M_i$ correct? Is it consistent with what is happening in `minnow`? I feel like I am missing something here, perhaps an $\epsilon$ term, or is it just a notation issue?

3. The GW corrections in e.g. https://arxiv.org/abs/2501.06990 are easily included by further augmenting the state-space to include $a(t)$. Obviously, we can also modify the state transition equations to use e.g. Ornstein-Uhlenbeck for the frequency evolution, two component model, whatever... 

# Appendix 

The numerical values of the $M$ columns in the `.feather` file is the same across all rows, e.g.

In [10]:
df["Mmat_0"].unique()

array([0.0053621])

### A1. Loading `.par` and `.tim` files

This section explores how `minnow` loads single band pulsar data, i.e. the `SingleBandPulsar` class in [pulsar.py](https://github.com/meyers-academic/minnow/blob/main/src/minnow/pulsar.py).

In [11]:
1+1

2

In [12]:
import numpy as np 

class SingleBandPulsar(): #note: I have removed the ds.Pulsar argument which depends on the Discovery package 
    """Single -- A class for handling pulsar data with a single frequency channels
    at given TOA.
    """
    def __init__(self, toas, residuals, radio_frequencies,
                 toaerrs, backend_flags, Mmat, fitpars,
                 noisedict=None, name='psr'):

        self.toas = toas
        self.toaerrs = toaerrs
        self.residuals = residuals
        self.radio_freqs = radio_frequencies
        self.backend_flags = backend_flags
        self.fitpars = fitpars

        self.toa_diffs = np.diff(toas)
        self.toa_diff_errors = np.sqrt(toaerrs[1:]**2 + toaerrs[:-1]**2)

        # cut up Mmat
        # Mmat = cutfunc(Mmat, fitpars)
        self.Mmat = Mmat
        if noisedict:
            self.noisedict = noisedict
        self.name = name


    @classmethod
    def read_par_tim(cls, p, t, **kwargs):
        return cls.from_enterprise(epPulsar(str(p), str(t), **kwargs))

    @classmethod
    def from_enterprise(cls, ds_psr):

        if hasattr(ds_psr, 'noisedict'):
            noisedict = ds_psr.noisedict
        else:
            noisedict = None
        return cls(ds_psr.toas, ds_psr.residuals,
                   ds_psr.freqs, ds_psr.toaerrs,
                   ds_psr.backend_flags, ds_psr.Mmat, ds_psr.fitpars,
                   noisedict=noisedict, name=ds_psr.name)

In [13]:
from enterprise.pulsar import Pulsar as epPulsar

psr = 'J0030+0451'

#I don't have access to the same par/tim files used in the test, so lets try with the IPTA mock data sets
par = f'../data/IPTA_MockDataChallenge/IPTA_Challenge1_open/Challenge_Data/Dataset1/{psr}.par'
tim = f'../data/IPTA_MockDataChallenge/IPTA_Challenge1_open/Challenge_Data/Dataset1/{psr}.tim'


psr_file = SingleBandPulsar.read_par_tim(par, tim, timing_package="pint", ephem="DE440", bipm_version="BIPM2019", clk="TT(BIPM2019)")

/fred/oz022/tkimpson/conda_envs/Argus3/lib/python3.12/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'EPHVER 5'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
2025-01-21 12:24:00.441 | WARNING  | pint.models.model_builder:__call__:229 - UNITS is not specified. Assuming TDB...
2025-01-21 12:24:00.486 | DEBUG    | pint.toa:__init__:1377 - No pulse number flags found in the TOAs
2025-01-21 12:24:00.488 | DEBUG    | pint.toa:apply_clock_corrections:2224 - Applying clock corrections (include_bipm = False)
2025-01-21 12:24:00.496 | INFO     | pint.observatory.topo_obs:clock_corrections:354 - Observatory axis requires no clock corrections.
2025-01-21 12:24:00.553 | DEBUG    | pint.toa:compute_TDBs:2270 - Computing TDB columns.
2025-01-21 12:24:00.553 | DEBUG    | pint.toa:compute_TDBs:2291 - Using EPHEM = DE440 for TDB calculation.
2025-01-21 12:24:00.572 | DEBUG    | pint.toa:get_TOAs:310 - Planet PosVels will be calculated.
2025

In [ ]:
psr_file.Mmat.shape # I guess this is 8 the design matrix, 8 columns for 8 parameters

(130, 8)

In [19]:
psr_file.residuals.shape

(130,)